In [306]:
#import data
import numpy as np
import pandas as pd
test_  = pd.read_csv(r"C:\Users\wkd72\Desktop\19-2\filedown\public_test.csv", encoding='UTF-8')
train = pd.read_csv(r"C:\Users\wkd72\Desktop\19-2\filedown\train.csv", encoding='UTF-8')

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
#import csv
#import sys
#import random
import re
import pprint
from gensim.models import Word2Vec
from konlpy.tag import Okt
from sklearn.decomposition import PCA

#%%
path = "C:\\Users\\wkd72\\Desktop\\19-2\\공모전\\text\\filedown"

#%%
data = pd.read_csv("{}/train.csv".format(path))
data = data[["text", "smishing"]]
test = pd.read_csv("{}/public_test.csv".format(path))
test.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


In [6]:
import random

random.seed(2050)
N= 20000
N_test = 2000
index = random.sample(range(1, data.shape[0]),N )
index_test = random.sample(range(1, data.shape[0]),N_test )


In [7]:
train = data.iloc[index,:].copy().reset_index(drop=True)
train_test = data.iloc[index_test,:].copy().reset_index(drop=True)

In [8]:
#%%

def preprocessing(text_data, okt, remove_stopwords=False, stop_words=[]):
    '''
    remove_stopwords: 불용어 제거 여부 선택
    stop_words: 제거할 불용어 리스트
    '''
    text_data_refined = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text_data)
    text_data_okt = okt.pos(text_data_refined, stem=True)
    # 불용어 제거 부분 추가
    return text_data_okt

#%%
# =============================================================================
# 한글만을 추출
# =============================================================================
okt = Okt()
clean_text = []
#for i in range(len(train_data["text"])):
for i in range(N):
    text = train["text"][i]
    if type(text) == str: # 비어있는 데이터인지 확인
        clean_text.append(preprocessing(text, okt))
    else:
        clean_text.append([]) # 비어있는 데이터
    #printProgress(i, len(train_data["text"]) - 1, "preprocessing: ", "Complete", 1, 50)

#%%
# =============================================================================
# 특정 품사만을 선택
# =============================================================================
useful_tag = ['Noun', 'Verb', 'Adje', 'Adve', 'Suffix']
clean_train = []
for i in range(len(clean_text)):
    clean_train.append([x for (x, y) in clean_text[i] if y in useful_tag])

In [9]:
#%%

def preprocessing(text_data, okt, remove_stopwords=False, stop_words=[]):
    '''
    remove_stopwords: 불용어 제거 여부 선택
    stop_words: 제거할 불용어 리스트
    '''
    text_data_refined = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text_data)
    text_data_okt = okt.pos(text_data_refined, stem=True)
    # 불용어 제거 부분 추가
    return text_data_okt

#%%
# =============================================================================
# 한글만을 추출
# =============================================================================
okt = Okt()
clean_text = []
#for i in range(len(train_data["text"])):
for i in range(N_test):
    text = train_test["text"][i]
    if type(text) == str: # 비어있는 데이터인지 확인
        clean_text.append(preprocessing(text, okt))
    else:
        clean_text.append([]) # 비어있는 데이터
    #printProgress(i, len(train_data["text"]) - 1, "preprocessing: ", "Complete", 1, 50)

#%%
# =============================================================================
# 특정 품사만을 선택
# =============================================================================
useful_tag = ['Noun', 'Verb', 'Adje', 'Adve', 'Suffix']
clean_testing = []
for i in range(len(clean_text)):
    clean_testing.append([x for (x, y) in clean_text[i] if y in useful_tag])

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train)
sequences = tokenizer.texts_to_sequences(clean_train)

X_data = sequences
y_train = train['smishing']
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 200
# 전체 데이터셋의 길이는 200으로 맞춥니다.
X_data = pad_sequences(X_data, maxlen=max_len,truncating = 'post')

word_to_index = tokenizer.word_index
vocab_size = len(word_to_index)+1
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 32, input_length = max_len)) # 임베딩 벡터의 차원은 32
model.add(LSTM(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_data, y_train, epochs=3, batch_size=128, validation_split=0.25)

Instructions for updating:
Colocations handled automatically by placer.
Train on 15000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
15000/15000 [==============================] - 32s 2ms/sample - loss: 0.0933 - acc: 0.9821 - val_loss: 0.0137 - val_acc: 0.9990
Epoch 2/3
15000/15000 [==============================] - 30s 2ms/sample - loss: 0.0083 - acc: 0.9993 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 3/3
15000/15000 [==============================] - 30s 2ms/sample - loss: 0.0051 - acc: 0.9991 - val_loss: 0.0020 - val_acc: 0.9998


In [71]:

#maximum 확인
maximum = 0
for i in range(N):
    temp = len(sequences[i])
    if(maximum < temp):
        maximum = temp
        
print(maximum)

294


In [104]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_testing)
sequences = tokenizer.texts_to_sequences(clean_testing)

X_testing = sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_testing = pad_sequences(X_testing, maxlen=max_len, truncating = 'post')

In [105]:
X_test = pad_testing
y_test = train_test['smishing']

In [110]:
model.evaluate(X_test, y_test)[1]

1000/1000 [==============================] - 1s 1ms/sample - loss: 0.2070 - acc: 0.9630


0.963

In [111]:
from sklearn import  metrics
result = model.predict(X_test)
print(metrics.roc_auc_score(y_test, result))

0.9514036016949153


In [ ]:
############

In [112]:
result = result.reshape(1000,)

In [113]:
haya = pd.DataFrame({'real': y_test, 'estimate' : result})

In [130]:
adverse_index = []
for i in range(N):
    if('광고' in clean_train[i]):
        adverse_index.append(i)

In [146]:
smishing_index = train[train['smishing']==1]['text'].index

True

In [158]:
flag = []
for i in range(len(smishing_index)):
    if(not '광고' in train[train['smishing']==1]['text'][smishing_index[i]]):
        flag.append(smishing_index[i])

In [159]:
flag

[1150,
 1302,
 1344,
 1636,
 1660,
 1885,
 1982,
 2279,
 2713,
 3223,
 3321,
 3972,
 4159,
 4361,
 4414,
 4461,
 4468,
 4519,
 4707,
 5704,
 5938,
 6041,
 6054,
 6250,
 6320,
 6619,
 6756,
 6783,
 7398,
 7809,
 7934,
 8053,
 8112,
 8417,
 8662,
 8708,
 8930,
 9219,
 9328,
 9601]

In [168]:
train.iloc[flag,:]['text'].index

Int64Index([1150, 1302, 1344, 1636, 1660, 1885, 1982, 2279, 2713, 3223, 3321,
            3972, 4159, 4361, 4414, 4461, 4468, 4519, 4707, 5704, 5938, 6041,
            6054, 6250, 6320, 6619, 6756, 6783, 7398, 7809, 7934, 8053, 8112,
            8417, 8662, 8708, 8930, 9219, 9328, 9601],
           dtype='int64')

In [176]:
train['text'][4414]

'XXX금전을 요구하는건 사기입니다. 유의하세요!1.부득이하게 높은 이자를 내고 있는 분2.월 불입금을 줄이고 하시는 분3.건수를 줄여 신용등급을 올리고 싶으신 분4.복잡한 부채를 하나로 묶어 관리하고자 하는 분수신을 원치 않으신분들이라면 거부라는 답장을 주시면 다시는 발송되지않습니다.기존 높은금리상품에서 저금리상품으로 금리전환하는걸 대환상품이라 하죠 이렇게 높은금리에서 저금리로 갈아타면 금리차이로 인해 이자가 절약됩니다.여기서 포인트는 바로 금리차이! 대환상품은 기존보다 낮은 금리로 갈아타는것이 매우 중요합니다.빚테크의 시작 대환상품을 통해 이자절감 받으세요XXX XXX 직장인상품적용대상: 만19세이상상품금리: 최저2.81%차등적용 자격조건: 현 직장에서 건강보험료 납부실적 6개월이상상품기간: 원리금균등방식(최소3년최대5년)               만기일시상환:1년(연장가능)(은행이 정한 기준에 따라 상품기간 및 상환방식이 제한이 될 수 있습니다)상품한도: 최고 1억4천만원까지(월소득의21배까지)(한도는 은행이 정한 산정방법에 의거 산출함)문자상담 신청방법문자로상담이라고 답장을 주시거나 전화를 주시면 최선을 다해 도와드리겠습니다.카톡 및 오픈채팅 신청방법 ID:  citi55  아이디 검색후 친구추가하시고 상담이라고 카톡주시면 됩니다. open.kakao.comosHLITO1 (입장하셔서 문의 남겨 주세요)부채통합(대환대출)상품대상 1년이상 재직중인 직장인금리 최저 3%차등적용한도 연봉대비 250%까지 한도발생자격조건 직장인이라면 누구나 가능 단 연체자는 불가합니다.원리금균등방식을 이자로 내는 방식으로 전환하여 월 최대 불입금을 최고 85%까지 줄일수 있습니다.최근 6개월간 연체사실만 없다면 누구나 진행이 가능하며 카드론현금서비스가 잦은 분들도 가능합니다.신용등급이 7등급XXX 분들중XXX'

In [118]:
check = train_test[haya['real']-haya['estimate']>0.9]['text']
print(check)

5      (광고)XXX  18년 2월부로 법정최고 금리가 낮아짐에 따라 평균 35프로 금리차...
44     (광고) XXX금융소비자 질문 TOP31. 가상조회는 신용등급에 영향이 없나요?소정...
60     (광고)XXX KoreaXXX기존 고금리대출에서 저금리대출로 대출갈아타기를 대환대출...
67     (광고) 안XXX요페퍼XXX에서XXX고객님분들 개인신용 상담및 실행업무를 맡고있는 ...
93     (광고)XXX저희 XXX에서 신용대출  안내 문자 드립니다.연봉대비 2 배 2.5배...
109    (광고)햇살론XXX따뜻한 햇살처럼XXX 고객님께 빛이 되는 XXX!정부지원조건이 일...
149    (광고)XXX고객지원팀 입니다.항상 이용해주셔서 감사합니다.2018년 하반기 정부지...
173    (광고)XXX 안내항상 XXX을 이용해주셔서 감사합니다.정부지원조건이 일부변경되어 ...
191    (광고)N H 농 협 고객지원팀 입니다항상 이용해주셔서 감사합니다 2017년 중반기...
298    (광고) 신협  상품 안내 안녕하십니까.저희 신협에서 신규 상품이 출시되어 채무통합...
352    XXX(광고)한국 XXX Korea(XXX)고금리 상품로 현재 힘이 든 사람?한달 ...
363    (광고)XXX1.여러곳을 알아봤지만추가 진행이 어려우신분2.높은 금리를 사용중이신분...
368    (광고) 날씨는 쌀쌀하지만 마음만큼은 따뜻하게 행복과행운이 가득한 오늘이 되길바랍니...
479    (광고)X XXX 안내문 (광고) XXX XXX 안내문     안녕하세요 XXX X...
525    (광고)따뜻한 햇살처럼XXX 고객님께 빛이 되는 XXX!정부지원조건이 일부변경되어 ...
533    (광고)XXX안녕하세요 XXX및XXX 담당자입니다 이번에 저희 은행에서직장인 신용대...
538    (광고)안녕하세요대성금융 XXX팀장입니다.상담:XXX-XXX-XXX카톡ID :goo...
546    (광고)XXX금융소비자 질문 TOP31. 가

In [117]:
check1 = train_test[haya['estimate']-haya['real']>0.5]['text']
print(check1)

447    XXX은행 리브똑똑 이벤트 당첨 안내XXX 고객님똑똑! 안녕하세요? 리브똑똑을 소개...
Name: text, dtype: object


In [179]:
train_test['text'][868]

'(XXX은행)(광고)지만 혜택되는 정보!본 메시지를 받으신 XXX평창동지점 우수 고객님께만 드리는 특별한 금리우대쿠폰!소중한 XXX 고객님 XXX은행 정기예금의 대표 브랜드인 XXX수퍼정기예금에 가입하시면 상품 우대금리 외에 추가로 연 0.3%p를 한시적으로 더 드립니다!(적용 시 연2%이상 -10월23일 기준)본  문자를 수신 받으신  우수 고객님께서 거래하시는  평창동지점에  방문하시어 아래의 조건에 따라 XXX수퍼정기예금  신규 시 상품우대금리 외에 추가로 금리우대를 받으실 수 있습니다! ㅇ 대상상품 : XXX수퍼정기예금(고정금리형-만기일시지급식)ㅇ 신규금액 : 5천만원이상1억원이하ㅇ 계약기간 : 12개월ㅇ 추가우대금리 : 연 0.3%pㅇ 쿠폰 유효기간 : 2017.9.27(수)  10.31(화)  (금리우대쿠폰은 유효기간 내  첫번째  신규계좌에 자동 적용되며 만기 해지 시에만 최종 적용됩니다)금리우대쿠폰은 문자수신고객의 고객식별자에 따라 적용되며 유효기간 내  XXX수퍼정기예금    고정금리형 만기일시지급식 최초 1회 신규계좌에 한해 적용됩니다.    (비대면채널(인터넷뱅킹 모바일뱅킹)을 통한 가입시  금리우대쿠폰은 적용되지 않습니다) 본 행사는 XXX은행의 영업정책이나 사정에 따라 변경 또는 중단될 수 있습니다. 기타 자세한 내용은 XXX은행 평창동지점 (XX-XXX-XXX) 로 문의 하시기 바랍니다. 무료 수신거부 XXX-XXX-XXX 준법감시인 심의필 제504호 (2017.07.04) 본 메시지의 유효기간 : 2017.10.31(화)'

In [ ]:
###########test

In [366]:
#%%

def preprocessing(text_data, okt, remove_stopwords=False, stop_words=[]):
    '''
    remove_stopwords: 불용어 제거 여부 선택
    stop_words: 제거할 불용어 리스트
    '''
    text_data_refined = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text_data)
    text_data_okt = okt.pos(text_data_refined, stem=True)
    # 불용어 제거 부분 추가
    return text_data_okt

#%%
# =============================================================================
# 한글만을 추출
# =============================================================================
okt = Okt()
clean_text_test = []
#for i in range(len(train_data["text"])):
for i in range(test.shape[0]):
    text = test["text"][i]
    if type(text) == str: # 비어있는 데이터인지 확인
        clean_text_test.append(preprocessing(text, okt))
    else:
        clean_text_test.append([]) # 비어있는 데이터
    #printProgress(i, len(train_data["text"]) - 1, "preprocessing: ", "Complete", 1, 50)

#%%
# =============================================================================
# 특정 품사만을 선택
# =============================================================================


In [367]:
useful_tag = ['Noun', 'Verb', 'Adje', 'Adve', 'Suffix']
clean_test = []
for i in range(len(clean_text_test)):
    clean_test.append([x for (x, y) in clean_text_test[i] if y in useful_tag])

###############

In [368]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_test)
sequences = tokenizer.texts_to_sequences(clean_test)

In [369]:
word_to_index = tokenizer.word_index
#print(word_to_index)
vocab_size = len(word_to_index)+1
print(vocab_size)

X_data = sequences

5636


In [370]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 150
# 전체 데이터셋의 길이는 200으로 맞춥니다.
data_test = pad_sequences(X_data, maxlen=max_len)

In [371]:
smishing_result = model.predict(data_test)
smishing_result = smishing_result.reshape(smishing_result.shape[0],)

In [372]:
smishing_result

array([0.00012481, 0.09851322, 0.0001353 , ..., 0.00015953, 0.00011337,
       0.00014754], dtype=float32)

In [373]:
result_excel = pd.DataFrame({'id' : test['id'], 'smishing' : smishing_result })

In [374]:
result_excel.to_csv("result.csv", index = False)

In [381]:
################
smishing_result

array([0.00012481, 0.09851322, 0.0001353 , ..., 0.00015953, 0.00011337,
       0.00014754], dtype=float32)

In [121]:
len_list = []
for i in range(N):
    len_list.append(len(clean_text[i]))

In [126]:
len_list = np.array(len_list)

In [137]:
len_list>10

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
import random

random.seed(2020)
N=3000
index = random.sample(range(1, data.shape[0]),N )


In [237]:
index = data1['smishing']==1
index = np.array(index)
index1 = data['smishing']==0
index1=np.array(index1)

In [238]:
smismi = data.iloc[index,:].copy().reset_index(drop=True)
unsmi = data.iloc[index1,:].copy().reset_index(drop=True)

In [246]:
random.seed(2020)
india = random.sample(range(0,unsmi.shape[0]), smismi.shape[0]*3)

In [249]:
smigol = data.iloc[india,:].copy().reset_index(drop=True)

In [251]:
smigol.to_csv("unsmishing.csv", index = False)
smismi.to_csv("smishing.csv", index = False)

In [250]:
smismi

,text,smishing
0,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1
1,(광고)XXX추가 XXX품 특판 안내문XXX 지점에서 취급하고 있는 여신 XXX...,1
2,(광고)XXX신용관리 XXX 알고 싶다나의 신용과 재무상태는 직접 관리해야지 누군가...,1
3,(광고)안녕하세요수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA ...,1
4,(광고)한국citi bank 나의 대출한도와 금리는? 대출때문에 고민하고 있다거나 ...,1
5,(광고)다사다난했던 병신()년을 보내고 새로운 정유()년에는 늘 좋은 일과 뜻하시는...,1
6,(광고)한국 (XXX XXX)2분의 시간만 투자하시면 월200만원 SAVE(절감)되...,1
7,(광고)한국citi bank나의 대출한도와 금리는?대출때문에 고민하고 있다거나 높은...,1
8,(광고)신용협동조합 안내XXX세요하루하루 힘든업무와 스트레스에 고생많으십니다여유시간...,1
9,(광고)신협안녕하십니까 하루하루 힘들고 스트레스 받는 대한민국에서 고생이 많으신데 ...,1


In [375]:
train

,text,smishing
0,활짝핀 웃음으로 행복한 한주 되세요XXX은행XXX올림,0
1,XXX 고객님 한 주를 시작하는 월요일!휴일의 즐거움을 벗삼아 새로운 맘으로 활기찬...,0
2,XXX동춘동거래 감사드립니다 추가 문의사항있으면 언제든 연락주세요 XXX올림,0
3,멋진고객님 감사합니다 새로운한주의시작월요일 힘차게시작하셔여국XXX,0
4,(광고)XXX 고객님XXX의평생금융파트너XXX은행에XXX 고객님께도움이 되는 정보를...,0
5,(광고)XXX 고객님어제는 방송3사에서 봄이온다평양공연을 했습니다.같은 민족이라는 ...,0
6,11월의 끝자락을 멋지게 마무리 하세요.XXX은행흑석동XXX올림,0
7,XXX 고객님 안녕하세요어제업무처리해드렸던 XXX차장입니다흐린아침이지만즐거운 주말을...,0
8,XXX 고객님동부 2스타 ELF SH-2호20150717 삼성중공업 현대차 보통주 ...,0
9,XXX 고객님업무처리를 도와드린XXX동두천지점 XXX 계장입니다XXX 저희지점과 꾸...,0


In [244]:
range(1,2)

range(1, 2)

In [157]:
len_thresh = []
for i in range(100):
    temp = train.iloc[len_list>i,:]
    len_thresh.append(sum(temp['smishing'])/len(temp['smishing']))

In [221]:
l1  = data['text'].apply(len)>250
l2 = data['text'].apply(len)<1000



In [222]:
inindex = np.array(l1*l2)
temp = data.iloc[inindex,:]

In [223]:
sum(temp['smishing'])/len(temp['smishing'])

0.22863460165969296

In [220]:
temp.to_csv("train_park1.csv",index = False)

In [160]:
pd.DataFrame(len_thresh)

,0
0,0.058686
1,0.058686
2,0.058706
3,0.058725
4,0.058784
5,0.058843
6,0.058922
7,0.059339
8,0.060419
9,0.062656
